### DataFrames basics

For a complete overview see the official documentation at `https://juliadata.github.io/DataFrames.jl/stable/`

In [1]:
using DataFrames

DataFrame construction

In [2]:
cars = DataFrame(brand = ["Volvo", "Volkswagen", "Škoda"], motor = [2.0, 1.6, 1.3], doors = [3, 5, 5])

,brand,motor,doors
,String,Float64,Int64
1,Volvo,2.0,3
2,Volkswagen,1.6,5
3,Škoda,1.3,5


indexing like a matrix - not so useful

In [3]:
cars[2:3, 1:2]

,brand,motor
,String,Float64
1,Volkswagen,1.6
2,Škoda,1.3


list of column names - you can use this to iterate over columns

In [4]:
names(cars)

3-element Array{String,1}:
 "brand"
 "motor"
 "doors"

column names as symbols

In [5]:
propertynames(cars)

3-element Array{Symbol,1}:
 :brand
 :motor
 :doors

getting a column - is a vector

In [6]:
cars.brand

3-element Array{String,1}:
 "Volvo"
 "Volkswagen"
 "Škoda"

In [7]:
cars[!,"brand"]

3-element Array{String,1}:
 "Volvo"
 "Volkswagen"
 "Škoda"

In [8]:
cars[!,:brand]

3-element Array{String,1}:
 "Volvo"
 "Volkswagen"
 "Škoda"

getting multiple columns - is a DataFrame

In [9]:
cars[:,[:brand, :motor]]

,brand,motor
,String,Float64
1,Volvo,2.0
2,Volkswagen,1.6
3,Škoda,1.3


getting a row - is a DataFrameRow

In [10]:
cars[1,:]

,brand,motor,doors
,String,Float64,Int64
1,Volvo,2.0,3


multiple rows - is a DataFrame

In [11]:
cars[1:2, :]

,brand,motor,doors
,String,Float64,Int64
1,Volvo,2.0,3
2,Volkswagen,1.6,5


adding a column to an existing df

In [12]:
cars[:,:country] = ["Sweden", "Germany", "Czech Republic"];
cars

,brand,motor,doors,country
,String,Float64,Int64,String
1,Volvo,2.0,3,Sweden
2,Volkswagen,1.6,5,Germany
3,Škoda,1.3,5,Czech Republic


adding a row to an existing df

In [13]:
push!(cars, ["Fiat", 1.0, 3, "Italy"])
push!(cars, ["Chrysler", 2.4, 5, "USA"])

,brand,motor,doors,country
,String,Float64,Int64,String
1,Volvo,2.0,3,Sweden
2,Volkswagen,1.6,5,Germany
3,Škoda,1.3,5,Czech Republic
4,Fiat,1.0,3,Italy
5,Chrysler,2.4,5,USA


produces a df that contains a summary for columns of the argument

In [14]:
describe(cars)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,brand,,Chrysler,,Škoda,5,,String
2,motor,1.66,1.0,1.6,2.4,,,Float64
3,doors,4.2,3,5.0,5,,,Int64
4,country,,Czech Republic,,USA,5,,String


Iterating over rows of a DataFrame

In [15]:
row_stats(r::DataFrameRow) = println("A $(r.brand) with $(r.doors) doors and a $(r.motor)l motor.")

row_stats (generic function with 1 method)

In [20]:
row_stats.(eachrow(cars));

A Volvo with 3 doors and a 2.0l motor.
A Volkswagen with 5 doors and a 1.6l motor.
A Škoda with 5 doors and a 1.3l motor.
A Fiat with 3 doors and a 1.0l motor.
A Chrysler with 5 doors and a 2.4l motor.


#### A small join example

In [17]:
brands = DataFrame(parent = ["Fiat", "Volkswagen", "Volkswagen", "Geely", "Fiat"], brand = ["Fiat", "Volkswagen", "Škoda", "Volvo", "Chrysler"])

,parent,brand
,String,String
1,Fiat,Fiat
2,Volkswagen,Volkswagen
3,Volkswagen,Škoda
4,Geely,Volvo
5,Fiat,Chrysler


In [18]:
cars = leftjoin(cars, brands, on = :brand)

,brand,motor,doors,country,parent
,String,Float64,Int64,String,String?
1,Volvo,2.0,3,Sweden,Geely
2,Volkswagen,1.6,5,Germany,Volkswagen
3,Škoda,1.3,5,Czech Republic,Volkswagen
4,Fiat,1.0,3,Italy,Fiat
5,Chrysler,2.4,5,USA,Fiat


Sorting

In [ ]:
sort(cars, [:doors, :motor], rev=true)

Filtering

In [ ]:
filter(r-> r[:doors] > 3 && r[:motor] < 2, cars)

### A simple ML experiment

Load experimental data, from multiple experiments, merge them into one big DataFrame, average performance over folds and look for the best model. The script that was used to generate the data using the `MLJ` package is `generate_dataframes.jl`.

This is a folder where the data is saved

In [ ]:
savepath = "./data"
files = readdir(savepath)

In [ ]:
using CSV # for data reading
using Statistics # for averaging
results = map(x->CSV.read(joinpath(savepath,x)), files)
show(results[1], allcols = true, splitcols=false)

#### Missing values

Any operation on a vector containing a `missing` value results in a `missing` value

In [ ]:
mean([1, 2, missing, 3])

Thats why we use skipmissing

In [ ]:
mean(skipmissing([1, 2, missing, 3]))

Now we have the DataFrames from individual experiments, let's join them together.

In [ ]:
resdf = vcat(results...)
show(resdf, splitcols=false)

Now aggregate them over folds.

In [ ]:
agdf = combine(groupby(resdf, [:dataset, :model, :parameters]), names(resdf, Not([:dataset, :model, :parameters])) .=> mean)
show(agdf, allrows=true, splitcols=false)

Where do the missing values come from?

In [ ]:
filter(r->ismissing(r[:cross_entropy]) || ismissing(r[:auc]), resdf)

For K=301, we want to have missings, but for other values, we just want to ignore the one missing value.

In [ ]:
missmean(x) = all(ismissing, x) ? missing : mean(skipmissing(x)) # this returns mean ignoring the missing elements, but if all elements of x are missing, it returns missing

In [ ]:
agdf = combine(groupby(resdf, [:dataset, :model, :parameters]), names(resdf, Not([:dataset, :model, :parameters])) .=> missmean)
agdf = agdf[!,Not(:fold_missmean)] # drop the means of folds
rename!(agdf, :cross_entropy_missmean => :cross_entropy) # rename the aggregated columns
rename!(agdf, :auc_missmean => :auc)
show(agdf, allrows=true, splitcols=false)

What is the best model on each dataset?

In [ ]:
combine(x->sort(x, :cross_entropy), groupby(agdf, [:dataset]), ungroup = false)

In [ ]:
combine(x->sort(x, :auc, rev=true), groupby(agdf, [:dataset]), ungroup = false) # revert sorting since bigger auc is better

### DataFramesMeta.jl

This package works on top of DataFrames and enables SQL-like queries. We can try to do the above in one query.

In [ ]:
using DataFramesMeta

Best average result in terms of cross entropy on iris dataset.

In [ ]:
@linq resdf |>
    where(.!ismissing.(:cross_entropy), :dataset.=="iris") |>
    by([:dataset, :model, :parameters], cross_entropy=mean(:cross_entropy)) |>
    orderby(:cross_entropy) |>
    select(:dataset, :model, :parameters, :cross_entropy)

Best average result in terms of AUC on crabs dataset.

In [ ]:
@linq resdf |>
    where(.!ismissing.(:auc), :dataset.=="crabs") |>
    by([:dataset, :model, :parameters], auc=mean(:auc)) |>
    orderby(-:auc) |>
    select(:dataset, :model, :parameters, :auc)